In [6]:
import json
import utils
from rasa_core.domain import Domain

DEFAULT_DOMAIN_PATH = "data/test_domains/default_with_slots.yml"
def default_domain():
    return Domain.load(DEFAULT_DOMAIN_PATH)

domain=default_domain()
req={
        'domain': domain.as_dict(),
        'next_action': 'my_action',
        'sender_id': 'default',
        'tracker': {
            'latest_message': {
                'entities': [],
                'intent': {},
                'text': None
            },
            'active_form': {},
            'latest_action_name': None,
            'sender_id': 'default',
            'paused': False,
            'followup_action': 'action_listen',
            'latest_event_time': None,
            'slots': {'name': None},
            'events': [],
            'latest_input_channel': None
        }
    }

utils.dump(req)

{
  "domain": {
    "config": {
      "store_entities_as_slots": true
    },
    "intents": [
      {
        "greet": {
          "use_entities": true
        }
      },
      {
        "default": {
          "use_entities": true
        }
      },
      {
        "goodbye": {
          "use_entities": true
        }
      },
      {
        "affirm": {
          "use_entities": true
        }
      },
      {
        "thank_you": {
          "use_entities": true
        }
      },
      {
        "change_bank_details": {
          "use_entities": true
        }
      },
      {
        "simple": {
          "use_entities": true
        }
      },
      {
        "hello": {
          "use_entities": true
        }
      },
      {
        "why": {
          "use_entities": true
        }
      },
      {
        "next_intent": {
          "use_entities": true
        }
      }
    ],
    "entities": [
      "name"
    ],
    "slots": {
      "name": {
        "type": "rasa_core.slots.

In [2]:
response = {
    "events": [
        {"event": "slot", "value": "rasa", "name": "name"}],
    "responses": [{"text": "test text",
                   "buttons": [{"title": "cheap", "payload": "cheap"}]},
                  {"template": "utter_greet"}]}
utils.dump(response)
response = {
        'events': [
            {'event': 'form', 'name': 'restaurant_form', 'timestamp': None},
            {'event': 'slot', 'timestamp': None,
             'name': 'requested_slot', 'value': 'cuisine'}
        ],
        'responses': [
            {'template': 'utter_ask_cuisine'}
        ]
    }
utils.dump(response)

{
  "events": [
    {
      "event": "slot",
      "value": "rasa",
      "name": "name"
    }
  ],
  "responses": [
    {
      "text": "test text",
      "buttons": [
        {
          "title": "cheap",
          "payload": "cheap"
        }
      ]
    },
    {
      "template": "utter_greet"
    }
  ]
}
{
  "events": [
    {
      "event": "form",
      "name": "restaurant_form",
      "timestamp": null
    },
    {
      "event": "slot",
      "timestamp": null,
      "name": "requested_slot",
      "value": "cuisine"
    }
  ],
  "responses": [
    {
      "template": "utter_ask_cuisine"
    }
  ]
}


In [13]:
from rasa_core.trackers import DialogueStateTracker, EventVerbosity
from rasa_core.events import (
    UserUttered, ActionExecuted, Restarted, ActionReverted,
    UserUtteranceReverted)
from rasa_core.actions.action import ACTION_LISTEN_NAME

tracker = DialogueStateTracker("default", domain.slots)
print(tracker.latest_message)
tracker.update(ActionExecuted(ACTION_LISTEN_NAME))
print(tracker.latest_message)
intent = {"name": "greet", "confidence": 1.0}
tracker.update(UserUttered("/greet", intent, []))
tracker.update(ActionExecuted("my_action"))
tracker.update(ActionExecuted(ACTION_LISTEN_NAME))
print(tracker.latest_message)

UserUttered(text: None, intent: {}, entities: [])
UserUttered(text: None, intent: {}, entities: [])
UserUttered(text: /greet, intent: {'name': 'greet', 'confidence': 1.0}, entities: [])


In [14]:
intent = {"name": "bye", "confidence": 1.0}
tracker.update(UserUttered("/bye", intent, []))
print(tracker.latest_message)

UserUttered(text: /bye, intent: {'name': 'bye', 'confidence': 1.0}, entities: [])


In [27]:
from rasa_core.interpreter import RasaNLUHttpInterpreter
from rasa_core.utils import EndpointConfig, AvailableEndpoints

text="找音乐会"
tracker = DialogueStateTracker("default", domain.slots)
endpoint = EndpointConfig("http://localhost:5000")
interpreter=RasaNLUHttpInterpreter(model_name="nlu", 
                                   project_name='chinese',
                                   endpoint=endpoint)
parse_data = interpreter.parse(text)
# print(parse_data)
tracker.update(UserUttered(text, parse_data["intent"],
                           parse_data["entities"], parse_data))
# store all entities as slots
for e in domain.slots_for_entities(parse_data["entities"]):
    tracker.update(e)

print("Logged UserUtterance - "
             "tracker now has {} events".format(len(tracker.events)))
print(tracker.latest_message)

Logged UserUtterance - tracker now has 1 events
UserUttered(text: 找音乐会, intent: {'name': 'search_concerts', 'confidence': 0.9683102369308472}, entities: [{'start': 1, 'end': 4, 'value': '音乐会', 'entity': 'place', 'confidence': 0.736472186969171, 'extractor': 'ner_crf'}])


In [30]:
from rasa_core_sdk.executor import CollectingDispatcher
from actions.procs.actions import ActionAboutDate
dispatcher=CollectingDispatcher()
ac=ActionAboutDate()
ac.run(dispatcher, tracker, domain)

UserUttered(text: 找音乐会, intent: {'name': 'search_concerts', 'confidence': 0.9683102369308472}, entities: [{'start': 1, 'end': 4, 'value': '音乐会', 'entity': 'place', 'confidence': 0.736472186969171, 'extractor': 'ner_crf'}])


[{'event': 'slot', 'name': 'response', 'timestamp': None, 'value': 'ok'}]

In [32]:
from rasa_core_sdk.executor import ActionExecutor
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

executor = ActionExecutor()
action_package_name='actions.procs'
executor.register_package(action_package_name)

INFO:rasa_core_sdk.executor:Registered function for 'action_joke'.
INFO:rasa_core_sdk.executor:Registered function for 'action_about_date'.
INFO:rasa_core_sdk.executor:Registered function for 'action_search_concerts'.
INFO:rasa_core_sdk.executor:Registered function for 'action_search_venues'.
INFO:rasa_core_sdk.executor:Registered function for 'action_show_concert_reviews'.
INFO:rasa_core_sdk.executor:Registered function for 'action_show_venue_reviews'.
INFO:rasa_core_sdk.executor:Registered function for 'my_custom_action'.
INFO:rasa_core_sdk.executor:Registered function for 'restaurant_form'.


In [34]:
def create_api_response(events, messages):
    return {
        "events": events if events else [],
        "responses": messages
    }

action_name='action_about_date'
action = executor.actions.get(action_name)
events = action(dispatcher, tracker, domain)
create_api_response(events, dispatcher.messages)

UserUttered(text: 找音乐会, intent: {'name': 'search_concerts', 'confidence': 0.9683102369308472}, entities: [{'start': 1, 'end': 4, 'value': '音乐会', 'entity': 'place', 'confidence': 0.736472186969171, 'extractor': 'ner_crf'}])


{'events': [{'event': 'slot',
   'name': 'response',
   'timestamp': None,
   'value': 'ok'}],
 'responses': [{'text': 'all done.'},
  {'text': 'all done.'},
  {'text': 'all done.'}]}

In [37]:
from rasa_core.events import deserialise_events
evs=deserialise_events(events)
for ev in evs:
    tracker.update(ev)

ERROR:rasa_core.trackers:Tried to set non existent slot 'response'. Make sure you added all your slots to your domain file.
